**Install ngrok**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/final web app model

/content/drive/MyDrive/Colab Notebooks/final web app model


In [3]:
pip install pyngrok

In [4]:
from flask import Flask, request, render_template
import joblib
import numpy as np
from pyngrok import ngrok

In [5]:
port_num = 5000

In [ ]:
app = Flask(__name__)

ngrok.set_auth_token("2qMXGhIVIU4Jw30Qgi63It3eBkM_7ns7dk9iJY4D1ogvDQyER")
public_url = ngrok.connect(port_num).public_url

model = joblib.load('model_ckd.pkl')

print(type(model))

def determine_severity(lab_name, value):
    # Define reference ranges and severity levels for each lab
    lab_ranges = {
        "SystolicBP": {"low": 90, "high": 120, "levels": {
                "low": "Mild (Hypotension)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Hypertension)",  # Above high range
            }},
        "DiastolicBP": {"low": 60, "high": 80, "levels":{
                "low": "Mild (Hypotension)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Hypertension)",  # Above high range
            }},
        "FastingBloodSugar": {"low": 70, "high": 100, "levels":{
                "low": "Mild (Hypoglycemia)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Hyperglycemia)",  # Above high range
            }},
        "HbA1c": {"low": 5.7, "high": 6.5, "levels":{
                "low": "Mild",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe",  # Above high range
            }},
        "SerumCreatinine": {"low": 0.5, "high": 1.2, "levels":{
                "low": "Mild",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe",  # Above high range
            }},
        "BUNLevels": {"low": 7, "high": 18, "levels":{
                "low": "Mild",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe",  # Above high range
            }},
        "ProteinInUrine": {"low": 150, "high": 150, "levels":{
                "low": "Normal",  # Below low range
                "high": "Severe",  # Above high range
            }},
        "ACR": {"low": 30, "high": 300, "levels":{
                "low": "Mild",  # Below low range
                "normal": "Moderately",  # Within range
                "high": "Severe",  # Above high range
            }},
        "SerumElectrolytesSodium": {"low": 135, "high": 145, "levels":{
                "low": "Mild (Hyponatermia)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Hypernatremia)",  # Above high range
            }},
        "SerumElectrolytesPotassium": {"low": 3.5, "high": 5, "levels":{
                "low": "Mild (Hypokalemia)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Hyperkalemia)",  # Above high range
            }},
        "SerumElectrolytesCalcium": {"low": 8.5, "high": 10.2, "levels":{
                "low": "Mild (Hypocalcemia)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Hypercalcemia)",  # Above high range
            }},
        "SerumElectrolytesPhosphorus": {"low": 2.5, "high": 4.5, "levels":{
                "low": "Mild (Hypophosphatemia)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Hyperphosphatemia)",  # Above high range
            }},
        "HemoglobinLevels": {"low": 12, "high": 17.5, "levels":{
                "low": "Mild (Anemia)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Polycythemia)",  # Above high range
            }},
        "CholesterolTotal": {"low": 200, "high": 240, "levels":{
                "low": "Normal",  # Below low range
                "normal": "Borderline High",
                "high": "Severe",  # Above high range
            }},
        "CholesterolLDL": {"low": 100, "high": 190, "levels":{
                "low": "Normal",  # Below low range
                "normal": "Borderline High",
                "high": "Severe",  # Above high range
            }},
        "CholesterolHDL": {"low": 40, "high": 60, "levels":{
                "low": "Mild (Anemia)",  # Below low range
                "normal": "Normal",  # Within range
                "high": "Severe (Polycythemia)",  # Above high range
            }},
        "CholesterolTriglycerides": {"low": 150, "high": 200, "levels":{
                "low": "Normal",  # Below low range
                "normal": "Borderline High",
                "high": "Severe",  # Above high range
            }}

    }

    if lab_name in lab_ranges:
        ranges = lab_ranges[lab_name]
        if value < ranges["low"]:
            return ranges["levels"]["low"]
        elif value > ranges["high"]:
            return ranges["levels"]["high"]
        else:
            return ranges["levels"]["normal"]  # or return None/False if normal should not be reported

    return None  # Lab not found in ranges

@app.route('/')
def home():
    return render_template("ckd.html")

print(f"to access the golobal link please click")
print(public_url)
@app.route('/predict',methods=['POST','GET'])

def predict():
    lab_results = {}  # Store lab results and their values
    for field_name, value in request.form.items():
        if field_name.startswith("lab_"):  # Assuming lab fields start with "lab_"
            lab_results[field_name[4:]] = int(float(value))
    int_features = [int(float(x)) for x in request.form.values()]
    final = np.array(int_features).reshape(1, -1)
    print(int_features)
    print(final)
    prediction = model.predict(final)[0]
    print(prediction)
    abnormal_labs = []
    for lab_name, value in lab_results.items():
        severity = determine_severity(lab_name, value)  # Call a helper function
        if severity:
            abnormal_labs.append((lab_name, value, severity))
    if prediction == 0:
        return render_template('ckd.html',pred='The Person Has Normal Kidney Function.', abnormal_labs = abnormal_labs)
    elif prediction == 1:
        return render_template('ckd.html',pred='The Person Has Abnormal Kidney function.', abnormal_labs = abnormal_labs)

app.run(port = port_num)

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
to access the golobal link please click
https://135617e01bf5.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to

[71, 0, 0, 0, 2, 31, 1, 5, 2, 1, 4, 0, 0, 0, 0, 0, 113, 83, 72, 9, 4, 25, 46, 0, 123, 137, 3, 10, 3, 16, 207, 85, 21, 212, 0, 0, 5, 1, 0, 0, 4, 7, 5, 8, 76, 0, 0, 1, 1, 5, 10]
[[ 71   0   0   0   2  31   1   5   2   1   4   0   0   0   0   0 113  83
   72   9   4  25  46   0 123 137   3  10   3  16 207  85  21 212   0   0
    5   1   0   0   4   7   5   8  76   0   0   1   1   5  10]]
1


INFO:werkzeug:127.0.0.1 - - [28/Oct/2025 07:22:00] "GET /.js/materialize.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Oct/2025 07:22:00] "GET /js/init.js HTTP/1.1" 404 -
